In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

In [2]:
# 1. 폰트 파일 경로 설정
font_path = r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\5_fonts\Pretendard-Medium.otf"

# 2. 폰트 이름 가져오기
font_name = fm.FontProperties(fname = font_path).get_name()
fm.fontManager.addfont(font_path)

# 3. Matplotlib의 기본 폰트로 설정
plt.rc("font", family = font_name)

# 4. 마이너스 기호 깨짐 방지
plt.rcParams["axes.unicode_minus"] = False

print(f"설정된 폰트 이름: {font_name}")

설정된 폰트 이름: Pretendard


In [3]:
# ── 1. 전력 데이터 ──────────────────────────────
df1 = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\서울본부_정렬.csv", encoding = "cp949")
df2 = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\남서울본부_정렬.csv", encoding = "cp949")

# 서울 전체 합산
df_power = pd.concat([df1, df2]).groupby(["기준일자", "기준시"])["전력사용량(MWh)"].sum().reset_index()

# datetime 생성 (기준시 1~24 → 1~24시로 변환)
df_power["datetime"] = pd.to_datetime(df_power["기준일자"]) + pd.to_timedelta(df_power["기준시"], unit="h")

# 24시 → 다음날 00:00 처리
is_24 = df_power["기준시"] == 24
df_power.loc[is_24, "datetime"] = pd.to_datetime(df_power.loc[is_24, "기준일자"]) + pd.Timedelta(days = 1)

# ── 2. 기상 데이터 ──────────────────────────────
dfw1 = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\raw\2023년 종관기상관측_서울.csv", encoding = "cp949")
dfw2 = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\raw\2024년 종관기상관측_서울.csv", encoding = "cp949")
df_weather = pd.concat([dfw1, dfw2], ignore_index = True)

df_weather["datetime"] = pd.to_datetime(df_weather["일시"])
df_weather = df_weather[["datetime", "기온(°C)", "강수량(mm)", "풍속(m/s)", "습도(%)", "일사(MJ/m2)", "전운량(10분위)"]]

# ── 3. 미세먼지 데이터 ─────────────────────────
dfm1 = pd.read_excel(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\2023 미세먼지 초미세먼지.xlsx")
dfm2 = pd.read_excel(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\2024 미세먼지 초미세먼지.xlsx")

# 2023: 24시 처리
dfm1["측정일시"] = dfm1["측정일시"].astype(str).str.strip()
is_24_m1 = dfm1["측정일시"].str.endswith("-24")
dfm1.loc[is_24_m1, "측정일시"] = dfm1.loc[is_24_m1, "측정일시"].str[:-2] + "00"
dfm1["datetime"] = pd.to_datetime(dfm1["측정일시"], format = "%Y-%m-%d-%H")
dfm1.loc[is_24_m1, "datetime"] += pd.Timedelta(days = 1)

# 2024: 24시 처리
dfm2["측정일시"] = dfm2["측정일시"].astype(str)
is_24_m2 = dfm2["측정일시"].str[-2:] == "24"
dfm2.loc[is_24_m2, "측정일시"] = dfm2.loc[is_24_m2, "측정일시"].str[:-2] + "00"
dfm2["datetime"] = pd.to_datetime(dfm2["측정일시"], format = "%Y%m%d%H")
dfm2.loc[is_24_m2, "datetime"] += pd.Timedelta(days = 1)

# 합치기
df_dust = pd.concat([dfm1, dfm2], ignore_index = True)
df_dust = df_dust.groupby("datetime")[["미세먼지(PM10)", "초미세먼지(PM25)"]].mean().reset_index()

# ── 4. 최종 merge ───────────────────────────────
df_final = df_power.merge(df_weather, on = "datetime", how = "left") \
                   .merge(df_dust, on = "datetime", how = "left")

# ── 5. 보간 (연말 누락 기상 데이터) ────────────
df_final["기온(°C)"] = df_final["기온(°C)"].interpolate(method = "linear")
df_final["습도(%)"] = df_final["습도(%)"].interpolate(method = "linear")
df_final["전운량(10분위)"] = df_final["전운량(10분위)"].interpolate(method = "linear")
df_final["풍속(m/s)"] = df_final["풍속(m/s)"].interpolate(method = "linear")

# ── 6. 저장 ────────────────────────────────────
df_final.to_csv(
    r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final.csv",
    index = False,
    encoding = "utf-8-sig"
)

print(df_final.shape)
print(df_final.isnull().sum())
print("저장 완료!")

(17544, 12)
기준일자               0
기준시                0
전력사용량(MWh)         0
datetime           0
기온(°C)             0
강수량(mm)        15506
풍속(m/s)            0
습도(%)              0
일사(MJ/m2)       8000
전운량(10분위)          0
미세먼지(PM10)         0
초미세먼지(PM25)        0
dtype: int64
저장 완료!


In [4]:
import holidays

df_final = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final.csv")

# datetime 변환
df_final["datetime"] = pd.to_datetime(df_final["datetime"])

# 시간 피처 추가
df_final["hour"]       = df_final["datetime"].dt.hour
df_final["dayofweek"]  = df_final["datetime"].dt.dayofweek
df_final["month"]      = df_final["datetime"].dt.month
df_final["is_weekend"] = df_final["dayofweek"].isin([5, 6]).astype(int)

# 공휴일 피처 추가
kr_holidays = holidays.KR(years = [2023, 2024])
df_final["is_holiday"] = df_final["datetime"].dt.date.apply(
    lambda x: 1 if x in kr_holidays else 0
)

# 주말 + 공휴일 통합 피처
df_final["is_off"] = ((df_final["is_weekend"] == 1) | (df_final["is_holiday"] == 1)).astype(int)

# 저장
df_final.to_csv(
    r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final_featured.csv",
    index = False,
    encoding = "utf-8-sig"
)

print(df_final.shape)
print(df_final.columns.tolist())
print("저장 완료!")

(17544, 18)
['기준일자', '기준시', '전력사용량(MWh)', 'datetime', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일사(MJ/m2)', '전운량(10분위)', '미세먼지(PM10)', '초미세먼지(PM25)', 'hour', 'dayofweek', 'month', 'is_weekend', 'is_holiday', 'is_off']
저장 완료!


In [5]:
df = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final_featured.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

def get_season(m):
    if m in [3, 4, 5]: return "봄"
    elif m in [6, 7, 8]: return "여름"
    elif m in [9, 10, 11]: return "가을"
    else: return "겨울"

df["season"] = df["month"].apply(get_season)

def dust_grade(v):
    if v <= 30: return "좋음"
    elif v <= 80: return "보통"
    elif v <= 150: return "나쁨"
    else: return "매우나쁨"

df["dust_grade"] = df["미세먼지(PM10)"].apply(dust_grade)

In [6]:
df = pd.read_csv(r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final_featured.csv")
df["datetime"] = pd.to_datetime(df["datetime"])

# ① CDD / HDD
df["CDD"] = (df["기온(°C)"] - 18).clip(lower=0)
df["HDD"] = (18 - df["기온(°C)"]).clip(lower=0)

# ② lag_24 / ③ lag_168
df["lag_24"]  = df["전력사용량(MWh)"].shift(24)
df["lag_168"] = df["전력사용량(MWh)"].shift(168)

# ④ rolling_24h
df["rolling_24h"] = df["전력사용량(MWh)"].shift(1).rolling(24).mean()

# ⑤ 체감온도
V_kmh = df["풍속(m/s)"] * 3.6
df["체감온도"] = df["기온(°C)"]  # 기본값 기온
winter_mask = (df["기온(°C)"] <= 10) & (df["풍속(m/s)"] >= 1.3)
df.loc[winter_mask, "체감온도"] = (
    13.12 
    + 0.6215 * df.loc[winter_mask, "기온(°C)"] 
    - 11.37 * (V_kmh[winter_mask]**0.16) 
    + 0.3965 * df.loc[winter_mask, "기온(°C)"] * (V_kmh[winter_mask]**0.16)
)

# ⑥ 불쾌지수
df["불쾌지수"] = 0.81 * df["기온(°C)"] + 0.01 * df["습도(%)"] * (0.99 * df["기온(°C)"] - 14.3) + 46.3

# ⑦ 태양광추정
df["태양광추정"] = df["일사(MJ/m2)"].fillna(0) * 0.2778 * 0.18

# lag/rolling NaN 제거
df = df.dropna(subset=["lag_24", "lag_168", "rolling_24h"])

# 저장
df.to_csv(
    r"C:\Users\User\Desktop\MyRepo\Portfolio\Portfolio_ver2\1_data\processed\df_final_v2.csv",
    index=False,
    encoding="utf-8-sig"
)

print(df.shape)
print(df.columns.tolist())
print("저장 완료!")

(17376, 26)
['기준일자', '기준시', '전력사용량(MWh)', 'datetime', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일사(MJ/m2)', '전운량(10분위)', '미세먼지(PM10)', '초미세먼지(PM25)', 'hour', 'dayofweek', 'month', 'is_weekend', 'is_holiday', 'is_off', 'CDD', 'HDD', 'lag_24', 'lag_168', 'rolling_24h', '체감온도', '불쾌지수', '태양광추정']
저장 완료!
